# Cramer-Rao Bound of the Selective Inversion Recovery Method
This notebook replicates 5<sup>th</sup> column of Tab. 1 in the paper [Rapid quantitative magnetization transfer imaging: utilizing the hybrid state and the generalized Bloch model](https://doi.org/10.48550/arXiv.2207.08259), i.e. it calculates the Cramer-Rao bound (CRB) of the selective inversion recovery (SIR) method with a turbo-spin-echo readout as described by [Dortch et al.](https://doi.org/10.1002/mrm.22928) and [Cronin et al.](https://doi.org/10.1016/j.mri.2020.01.014)

First, we load all required packages:

In [1]:
using MRIgeneralizedBloch
using MRIgeneralizedBloch: propagator_linear_inversion_pulse, xs_destructor
using LinearAlgebra
using StaticArrays

Define the point in parameter space at which the CRB will be calculated:

In [2]:
m0s = 0.1
R1a = 0.625 # 1/s
R1f = R1a   # 1/s
R1s = R1a   # 1/s
R2f = 15    # 1/s
Rx  = 30    # 1/s
T2s = 10e-6 # s
R2slT = precompute_R2sl()

ω0 = 0    # rad/s
B1 = 1;   # in units of B1_nominal

The following cell defines the function that simulates the SIR signal for a given delay time `TD` and inversion time `TI` (cf. [Dortch et al.](https://doi.org/10.1002/mrm.22928) for details). In this simulation, we ignore the $B_1$-dependence of the semi-solid spin pool, which is achieved by the line `u_ip[9:10,4:5] .= 0`. Comment out this line to calculate the CRB while accounting for this dependence. Beware, though, of numerical instabilities at `B1 = 1` that might lead to large negative CRB values. Choose a different $B_1$ value in the cell above for more meaningful CRB values while accounting for this dependency. 

In [3]:
function simulate_SIR(TD, TI)
    _m0 = @SVector [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

    J = Vector{Float64}(undef, length(grad_list)+1)

    R2sl_inv = R2slT[1](TRF, π, B1, T2s)
    dR2sldT2s_inv = R2slT[2](TRF, π, B1, T2s)
    dR2sldB1_inv  = R2slT[3](TRF, π, B1, T2s)
    for j ∈ eachindex(grad_list)
        u_td = exp(hamiltonian_linear(0, B1, ω0, TD - TRF/2, m0s, R1f, R2f, Rx, R1s, 0, 0, 0, grad_list[j]))
        u_ip = propagator_linear_inversion_pulse(π/TRF, TRF, B1, R2sl_inv, dR2sldT2s_inv, dR2sldB1_inv, grad_list[j])

        u_ip = Matrix(u_ip)
        u_ip[9:10,4:5] .= 0 # remove B1-dependence of the semi-solid spin pool

        u_xs = xs_destructor(grad_list[j])
        u_ir = exp(hamiltonian_linear(0, B1, ω0, TI - TRF/2, m0s, R1f, R2f, Rx, R1s, 0, 0, 0, grad_list[j]))
        m = u_ir * (u_xs * (u_ip * (u_td * _m0)))
        J[1]   = m[3] # z-magnetization right before the excitation pulse
        J[j+1] = m[8] # its derivate wrt. grad_list[j]
    end
    return J
end;

Define the sequence settings according to [Dortch et al.](https://doi.org/10.1002/mrm.22928):

In [4]:
TRF = 1e-3; # duration of inversion pulse in s
TI = [  10,   50,   56,  277, 843] .* 1e-3  # inversion times in s
TD = [3270, 4489, 1652, 2922,  10] .* 1e-3; # delay times in s

Define the gradients that are accounted for in the CRB calculation:

In [5]:
grad_list = [grad_m0s(), grad_R1a(), grad_Rx(), grad_B1()];

Calculate the Jacobian matrix and the CRB values of all considered parameters:

In [6]:
J = Matrix{Float64}(undef, length(TI), length(grad_list)+1)
for i ∈ eachindex(TI, TD)
    J[i,:] .= simulate_SIR(TD[i], TI[i])
end
CRB = diag(inv(J' * J));

The following cell normalizes the CRB values by the scan time. Like for the other cases in Tab. 1, we consider each k-space line as a single data point with unit noise variance (this is equivalent to a normalization with the noise variance). 

In [7]:
Nlines_per_shots = 22 # phase encoding
EchoSpacing = 5.9e-3  # s
T_RO = Nlines_per_shots * EchoSpacing # readout time in s
CRB .*= (sum(TD) + sum(TI) + length(TI) * T_RO) ./ Nlines_per_shots;

Normalized CRB of $m_0^s$ in seconds:

In [8]:
CRB[2]/m0s^2

169.61697239948023

Normalized CRB of $R_1^f$ in seconds:

In [9]:
CRB[3]/R1f^2


25.558010375815314

Normalized CRB of $M_0$ in seconds:

In [10]:
CRB[1]


22.074108948862666

Normalized CRB of $R_x$ in seconds:

In [11]:
CRB[4]/Rx^2


1047.7250000622794

Normalized CRB of $B_1$ in seconds:

In [12]:
CRB[5]/B1^2

9.4597855622584e31